In [ ]:
# Get data from GBQ
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    '/Users/jaroslawbontruk/Documents/[name_of_file].json',
)

project_id = "[name_of_project]"

query = """
select supplier_name 
from [name_of_db].v_supplier_summary
where [name_of_column] = 'Active'
"""

df = pandas_gbq.read_gbq(query, 
                         project_id = project_id, 
                         credentials = credentials)

In [ ]:
# Prepare URLs (based on GBQ data)
sample_url = "https://www.sellerratings.com/amazon/usa?name=Soul+Sistahs+Network"
core_url = "https://www.sellerratings.com/amazon/usa?name="

df = df.apply(lambda x: x.str.replace(' ','+'))
df['url'] = core_url + df['supplier_name'].astype(str)

print(df.loc[10, 'url'])
print(df.shape)

In [ ]:
# Get data from CSV
import pandas as pd
import numpy as np

df1 = pd.read_csv('sheet.csv', low_memory = False)
df1.columns = ['all_suppliers', 'selected_suppliers', 'match']

grouped = df1.groupby('selected_suppliers').size().reset_index()
grouped.columns = ['selected_suppliers', 'count']
grouped = grouped.sort_values('count', ascending=False)

In [ ]:
# Prepare URLs (based on CSV data)
uniques = df1.selected_suppliers.unique()
uniques = pd.DataFrame(data = uniques.flatten())
uniques.columns = ['company_name_cleaned']
a = uniques['company_name_cleaned']

uniques = uniques.apply(lambda x: x.str.replace(',',''))
uniques = uniques.apply(lambda x: x.str.replace('!',''))
uniques = uniques.apply(lambda x: x.str.replace("'",""))
uniques = uniques.apply(lambda x: x.str.replace('.',''))
uniques = uniques.apply(lambda x: x.str.replace('&',''))
uniques = uniques.apply(lambda x: x.str.replace(' ','-'))

core_url = "https://www.sellerratings.com/amazon/usa/"
uniques['url'] = core_url + uniques['company_name_cleaned'].astype(str)
uniques['company_name'] = a

print(uniques.loc[29223, 'company_name_cleaned'])
print(uniques.shape)

df = uniques
df

In [ ]:
# Get data from Marketplace Pulse
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import json

results = pd.DataFrame(columns=['supplier_name', 'url', 'response', 'sellerID', 'reviewCount', 'ratingValue'])
start = time.time()

for i in range(29200, 30881):
    results.loc[i, 'supplier_name'] = df.loc[i, 'company_name']  
    results.loc[i, 'url'] = df.loc[i, 'url'].lower() 
    r = requests.get(df.loc[i, 'url'])
    results.loc[i, 'response'] = str(r)
    if str(r) == '<Response [200]>':
        soup = BeautifulSoup(r.content, 'html.parser')
        for div in soup.find_all(class_='contact'):
            sellerID = div.find('a')['href']
        sellerID = sellerID.replace('https://www.amazon.com/gp/help/contact-seller/contact-seller.html?sellerID=', '')        
        results.loc[i, 'sellerID'] = sellerID
        data = json.loads(soup.find('script', type='application/ld+json').text)
        results.loc[i, 'reviewCount'] = data['aggregateRating']['reviewCount']
        results.loc[i, 'ratingValue'] = data['aggregateRating']['ratingValue']
    if i%10 == 0: print(str(i) + ' URLs in ' + str(time.time() - start) + ' sec')
        
results

In [ ]:
# Save to CSV
results.to_csv('results.csv', index = False)